In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline



In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.mnist_classifier = nn.Sequential(
            nn.Conv2d(1, 8, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.05),
            nn.Conv2d(8, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05),
            nn.Conv2d(16, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Dropout(0.1),
            nn.Conv2d(16, 10, 3, bias=False),
            nn.AvgPool2d(2)
            

            
        )
    
    def forward(self, x):
        x = self.mnist_classifier(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         MaxPool2d-8           [-1, 16, 12, 12]               0
           Dropout-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 16, 10, 10]           2,304
             ReLU-11           [-1, 16, 10, 10]               0
      BatchNorm2d-12           [-1, 16, 10, 10]              32
          Dropout-13           [-1, 16, 10, 10]               0
           Conv2d-14             [-1, 1

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([

                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [5]:

train_losses = []
test_losses = []
train_acc = []
test_acc = []
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):

    model.train()
    correct_train = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        train_pred = output.argmax(dim = 1,keepdim = True)
        correct_train += train_pred.eq(target.view_as(train_pred)).sum().item()
        
        for param in optimizer.param_groups:
          lr = param['lr']
        pbar.set_description(desc= f'epoch = {epoch + 1} loss={loss.item()} batch_id={batch_idx} learning_rate={lr} train_accuracy={correct_train * 100.0/len(train_loader.dataset)}')
    train_acc.append(correct_train * 100.0/len(train_loader.dataset))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    test_acc.append(100. * correct / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch + 1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


epoch = 1 loss=0.07081147283315659 batch_id=468 learning_rate=0.01 train_accuracy=95.565: 100%|█| 469/469 [00:10<00:00,
  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0472, Accuracy: 9835/10000 (98%)

EPOCH: 2


epoch = 2 loss=0.024516845121979713 batch_id=468 learning_rate=0.01 train_accuracy=98.26666666666667: 100%|█| 469/469 [
  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

...

epoch = 10 loss=0.009639720432460308 batch_id=468 learning_rate=0.01 train_accuracy=99.02: 100%|█| 469/469 [00:10<00:00
  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0361, Accuracy: 9892/10000 (99%)

EPOCH: 11


epoch = 11 loss=0.013205687515437603 batch_id=468 learning_rate=0.01 train_accuracy=99.01833333333333: 100%|█| 469/469 
  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0254, Accuracy: 9917/10000 (99%)

EPOCH: 12


epoch = 12 loss=0.06937472522258759 batch_id=468 learning_rate=0.01 train_accuracy=99.10333333333334: 100%|█| 469/469 [


In [ ]:

print(f'max train_acc: {max(train_acc)}; max_test_acc : {max(test_acc)}')

fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(train_losses)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc)
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")